<a href="https://colab.research.google.com/github/Vakurin/SuperResolutionGAN-TF2.0/blob/master/SRGAN_TF2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pipeline SRGAN Training
---



In [0]:
# Make default version tf2.0 in colab
%tensorflow_version 2.x 
!pip install tensorlayer 

import tensorlayer as tl

import os
import tensorflow as tf
from tensorflow.keras.metrics import Mean

#import scipy
import numpy as np
import time

In [2]:
!git clone https://github.com/Vakurin/SuperResolutionGAN-TF2.0.git
%cd SuperResolutionGAN-TF2.0/

from models import model_vgg19, model_discriminator, model_generator
from dataset import get_train_data

Cloning into 'SuperResolutionGAN-TF2.0'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 17 (delta 3), reused 16 (delta 2), pack-reused 0
Unpacking objects: 100% (17/17), done.
/content/SuperResolutionGAN-TF2.0
TensorFlow Version: 2.1.0


## Config

In [0]:
IMG_HR_SIZE = 256
IMG_LR_SIZE = 64
CHANNELS = 3

BATCH_SIZE = 4
EPOCHS = 320

PATH_PRE_GENERATOR       = '/content/drive/My Drive/SRGAN/models/pre-train-model-3000.h5'

PATH_FOR_SAVE_MODELS     = '/content/drive/My Drive/SRGAN/models/'
PATH_FOR_SAVE_GEN_IMAGES = '/content/drive/My Drive/SRGAN/photos/'

PATH_FOLDER_WITH_IMAGES  = '/tmp/Dataset/DIV2K_train_HR/'

# Directory For TensorBoard
train_summary_writer = tf.summary.create_file_writer('logs/train')

##(Optional for colab) Dataset Unzipping 

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
import zipfile

zip_ref = zipfile.ZipFile("/content/drive/My Drive/SRGAN/DIV2K_train_HR.zip", 'r') # ~3 min
zip_ref.extractall("/tmp/Dataset")
zip_ref.close()

print('total training test images:', len(os.listdir(PATH_FOLDER_WITH_IMAGES)))

total training test images: 800


## Load Dataset

In [15]:
# Load All Files By Batch 4 
train_ds = get_train_data(IMG_HR_SIZE, IMG_LR_SIZE, BATCH_SIZE, PATH_FOLDER_WITH_IMAGES)

[TL] Match file list = ['0480.png', '0167.png', '0791.png', '0527.png', '0350.png', '0247.png', '0181.png', '0778.png', '0171.png', '0708.png', '0515.png', '0198.png', '0617.png', '0800.png', '0028.png', '0311.png', '0459.png', '0447.png', '0734.png', '0159.png', '0084.png', '0388.png', '0081.png', '0702.png', '0319.png', '0003.png', '0553.png', '0398.png', '0109.png', '0080.png', '0636.png', '0180.png', '0402.png', '0545.png', '0741.png', '0682.png', '0027.png', '0202.png', '0252.png', '0635.png', '0568.png', '0524.png', '0271.png', '0016.png', '0144.png', '0686.png', '0065.png', '0600.png', '0680.png', '0079.png', '0130.png', '0355.png', '0561.png', '0672.png', '0465.png', '0449.png', '0155.png', '0501.png', '0153.png', '0328.png', '0476.png', '0034.png', '0301.png', '0442.png', '0396.png', '0177.png', '0222.png', '0098.png', '0364.png', '0205.png', '0259.png', '0064.png', '0059.png', '0689.png', '0587.png', '0227.png', '0652.png', '0785.png', '0218.png', '0722.png', '0750.png', '004

## Model Inialization

In [0]:
gan_generator = model_generator()
gan_generator.load_weights(PATH_PRE_GENERATOR)

discriminator = model_discriminator()

# 5 or 20
VGG = model_vgg19(output_layer=20)

## Save Checkpoint

In [0]:
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=gan_generator,
                                 discriminator=discriminator)


checkpoint_manager = tf.train.CheckpointManager(checkpoint=checkpoint,
                                                    directory=checkpoint_dir,
                                                    max_to_keep=3)

## Losses
**Generator**  -- We used Perceptual Loss, It's Sum Of Two Function: 
* Content Loss: 1) Pixel-wise MSE Loss. 2) VGG Loss
* Adversarial loss - It's probability of Discriminator

In [0]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# G Loss
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)


# D Loss
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

# Content Loss (VGG 19)
@tf.function
def _content_loss(vgg_model, gen_fake, hr):
  gen_fake = (gen_fake + 1) / 2. # VGG19 used [0, 1]
  hr = (hr + 1) / 2.
  gen_fake_features = vgg_model(gen_fake) 
  hr_features = vgg_model(hr) 

  return tf.keras.losses.MeanSquaredError()(hr_features, gen_fake_features)

# Optimizers
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

## Training Steps 

In [0]:
@tf.function
def train_step(lr, hr):
  """
      lr - low resolution image x
      hr - high resolution image y 
  """

  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    generated_images = gan_generator(lr, training=True)
    real_output = discriminator(hr, training=True)
    fake_output = discriminator(generated_images, training=True)
    
    # G loss
    mse_loss = tf.keras.losses.MeanSquaredError()(generated_images, hr)
    content_loss  = _content_loss(VGG, generated_images, hr)
    gen_loss = generator_loss(fake_output)
  
    perceptual_loss = content_loss * 1e-3 + 2e-6 * gen_loss + mse_loss

    # D loss
    disc_loss = discriminator_loss(real_output, fake_output)
    
  # Backprop for G
  gradients_of_generator = gen_tape.gradient(perceptual_loss, gan_generator.trainable_variables)
  generator_optimizer.apply_gradients(zip(gradients_of_generator, gan_generator.trainable_variables))
  
  # Backprop for D
  gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
  discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

  return disc_loss, perceptual_loss, generated_images

In [0]:
def train(dataset, epochs, batch_size, writer):
  perceptual_metric = Mean()
  disc_metric = Mean()

  n_step_epoch = round(epochs // batch_size)

  for epoch in range(n_step_epoch):
    start = time.time()
    for step, (lr, hr) in enumerate(train_ds):
      disc_loss, perceptual_loss, gen_images = train_step(lr, hr)
      
      perceptual_metric(perceptual_loss)
      disc_metric(disc_loss)
      
    # Save Model Every 10 Epochs
    if (epoch) % 3 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)  
      
      tl.vis.save_images(gen_images.numpy(), [2, 4], os.path.join(PATH_FOR_SAVE_GEN_IMAGES, 'train_g_{}.png'.format(epoch)))
      
      gan_generator.save(os.path.join(PATH_FOR_SAVE_MODELS, 'generator-{}.h5'.format(epoch)))
      discriminator.save(os.path.join(PATH_FOR_SAVE_MODELS, 'discriminator-{}.h5'.format(epoch)))
      print('P Loss', perceptual_metric.result())
      print('D Lost', disc_metric.result())

      perceptual_metric.reset_states()
      disc_metric.reset_states()
      
    print ('Time for epoch {} of {} is {} sec'.format(epoch + 1, n_step_epoch, int(time.time()-start)))
  gan_generator.save(os.path.join(PATH_FOR_SAVE_MODELS, 'generator-final.h5'))
  discriminator.save(os.path.join(PATH_FOR_SAVE_MODELS, 'discriminator-final.h5'))  

## Training

In [50]:
train(train_ds, EPOCHS, BATCH_SIZE, train_summary_writer)

P Loss tf.Tensor(18.922638, shape=(), dtype=float32)
D Lost tf.Tensor(0.31444302, shape=(), dtype=float32)
Time for epoch 1 of 80 is 104 sec
Time for epoch 2 of 80 is 100 sec
Time for epoch 3 of 80 is 99 sec
P Loss tf.Tensor(18.569492, shape=(), dtype=float32)
D Lost tf.Tensor(0.2575065, shape=(), dtype=float32)
Time for epoch 4 of 80 is 103 sec
Time for epoch 5 of 80 is 100 sec
Time for epoch 6 of 80 is 100 sec
P Loss tf.Tensor(18.01409, shape=(), dtype=float32)
D Lost tf.Tensor(0.00516451, shape=(), dtype=float32)
Time for epoch 7 of 80 is 104 sec
Time for epoch 8 of 80 is 101 sec
Time for epoch 9 of 80 is 101 sec
P Loss tf.Tensor(18.182432, shape=(), dtype=float32)
D Lost tf.Tensor(0.2719872, shape=(), dtype=float32)
Time for epoch 10 of 80 is 106 sec
Time for epoch 11 of 80 is 101 sec
Time for epoch 12 of 80 is 101 sec
P Loss tf.Tensor(17.701242, shape=(), dtype=float32)
D Lost tf.Tensor(0.08752858, shape=(), dtype=float32)
Time for epoch 13 of 80 is 105 sec
Time for epoch 14 of 80

KeyboardInterrupt: ignored